In [2]:
###predict where the link will be posted given just the title!
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

train = pd.read_csv('../code/jan_may_2017_links_cross_posting_info.csv')
all_pages = list(train.buzz_account_display_name.unique())

In [3]:
train.columns

Index(['buzz_external_id', 'buzz_post_parent_external_id',
       'buzz_post_created_at', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name', 'ext_table1_stats_date', 'ext_external_id',
       ...
       'Tasty One-Pot', 'BuzzFeed Sweaty', 'Tasty Junior',
       'Oh Great, More Politics', 'Einfach Tasty', 'BuzzFeed Steven L.',
       'The Try Guys', 'Ohmygod Yaaa', 'BuzzFeed Chloe', 'Nifty Science'],
      dtype='object', length=113)

In [4]:
starting_features = ['buzz_external_id', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name',]
df = train[starting_features]

In [5]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['buzz_post_buzz_id']).agg({'buzz_post_created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'buzz_post_created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['buzz_post_buzz_id'])

df = df[df['buzz_post_created_at'] == df['first_post']]

In [6]:
###predict where the link will be posted given just the title!

In [14]:
###predict where the link will be posted given just the title!
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []
cv_folds = 10
results = pd.DataFrame()
page_clf = []

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.buzz_account_display_name == test_page]
    X_0 = df[~(df.buzz_account_display_name == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['buzz_campaign_name', 'buzz_account_display_name']]
    X_0 = X_0[['buzz_campaign_name', 'buzz_account_display_name']]

    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['buzz_account_display_name'].apply(get_class, page = test_page)

    X_all = X_all.dropna()

    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
#                          ('featselect', SelectKBest(k = 100)),
                         ('clf', MultinomialNB()),
#                          ('clf', LinearSVC()),
#                          ('clf', RandomForestClassifier()),
#                          ('clf', LogisticRegression()),
    ])

    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['buzz_campaign_name']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
    print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

Page:  BuzzFeed Quiz #links:  1316 Accuracy:  0.876523793064 Precision:  0.89950138848 Recall:  0.877731162861 Fscore: 0.874441455809
Page:  BuzzFeed Japan #links:  966 Accuracy:  0.778783184659 Precision:  0.868114769689 Recall:  0.791954328994 Fscore: 0.748948276832
Page:  BuzzFeed UK #links:  560 Accuracy:  0.705357142857 Precision:  0.781157461487 Recall:  0.708059582881 Fscore: 0.683895185497
Page:  BuzzFeed Brasil #links:  1022 Accuracy:  0.954043519847 Precision:  0.956977972847 Recall:  0.955607314823 Fscore: 0.953718606377
Page:  BuzzFeed Japan News #links:  616 Accuracy:  0.814221085759 Precision:  0.884922872754 Recall:  0.824908477568 Fscore: 0.790054704583
Page:  BuzzFeed Deutschland #links:  581 Accuracy:  0.963881520778 Precision:  0.963885637376 Recall:  0.96701080126 Fscore: 0.963645236684
Page:  BuzzFeed News #links:  649 Accuracy:  0.798151460942 Precision:  0.819450358049 Recall:  0.800452731514 Fscore: 0.794210492173
Page:  BuzzFeed #links:  978 Accuracy:  0.633417

In [15]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Quiz,1316,0.876524,0.899501,0.877731,0.874441
1,BuzzFeed Japan,966,0.778783,0.868115,0.791954,0.748948
2,BuzzFeed UK,560,0.705357,0.781157,0.708060,0.683895
3,BuzzFeed Brasil,1022,0.954044,0.956978,0.955607,0.953719
4,BuzzFeed Japan News,616,0.814221,0.884923,0.824908,0.790055
5,BuzzFeed Deutschland,581,0.963882,0.963886,0.967011,0.963645
6,BuzzFeed News,649,0.798151,0.819450,0.800453,0.794210
7,BuzzFeed,978,0.633417,0.698705,0.633948,0.600070
8,BuzzFeed Canada,251,0.706902,0.765658,0.706150,0.685729
9,BuzzFeed Video,121,0.631500,0.742402,0.636180,0.577839


In [22]:
results.mean()

num_links    357.557692
accuracy       0.775999
precision      0.823982
recall         0.781001
fscore         0.760379
dtype: float64

In [23]:
page_clf

[Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='c

In [17]:
#####Query by page and title of link - Should a link with this title be posted on this BF page?
# q_page = "LOLA"
q_page = "BuzzFeed Quiz"
# q_page = "BuzzFeed Japan"
q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

for pg in all_pages:
    try:
        clf = page_clf[pages.index(pg)]
    except:
        print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page)

BuzzFeed Quiz 0.872023265736
BuzzFeed Japan 0.0232061402676
BuzzFeed UK 0.639373371345
BuzzFeed Brasil 0.0159616094542
BuzzFeed Japan News 0.0546786379889
BuzzFeed Deutschland 0.0392927380669
BuzzFeed News 0.344790621089
BuzzFeed 0.720540555956
BuzzFeed Canada 0.688683690492
BuzzFeed Video 0.441653668603
Cocoa Butter 0.559871552776
BuzzFeed Oz Politics 0.29647916799
Quizzes En Español 0.165538767967
Obsessed by BuzzFeed 0.512451840258
BuzzFeed News BR 0.077149748131
Buy Me That 0.458684628105
BuzzFeed UK Politics 0.418383819611
BuzzFeed Australia 0.666488964193
BuzzFeed Rewind 0.5358195718
Woops, the model for page  BuzzFeed Ladylike  is still in the works!
BuzzFeed Books 0.658604598769
BuzzFeed DIY 0.557135392623
BuzzFeed México 0.0642621895175
BuzzFeed Food 0.609404586826
BuzzFeed Celeb 0.648610538258
BuzzFeed Animals 0.805783224629
BuzzFeed France 0.0500671445182
BuzzFeed Español 0.0780864748339
BuzzFeed Style 0.495625910281
Woops, the model for page  Tasty  is still in the works!
C

In [18]:
max_page, max_prob

('BuzzFeed Quiz', 0.87202326573578171)

In [28]:
X_all

,buzz_campaign_name,buzz_account_display_name,class
1474,Caltech Professor Who Harassed Women Was Also ...,BuzzFeed Science,1
1631,The CEO Of A Huge Chicken Company Denied That ...,BuzzFeed Science,1
1670,A NASA Spacecraft Has Found Northern Lights An...,BuzzFeed Science,1
1782,Scientists Tell Navajo Farmers Their Water Is ...,BuzzFeed Science,1
2218,"Why Americans Are So Damn Unhealthy, In 4 Shoc...",BuzzFeed Science,1
3606,This Is Why You Should Always Shower Before Sw...,BuzzFeed Science,1
4187,A Photographer Captured A Jaw-Dropping Weather...,BuzzFeed Science,1
7874,A Shocking 51-49 Senate Vote Just Kept Obama's...,BuzzFeed Science,1
7937,A Scientist Is Leaving Google For A Mental Hea...,BuzzFeed Science,1
8190,Obama Warns Inequality Could Derail Climate Ch...,BuzzFeed Science,1


In [19]:
dr = train[train['buzz_post_buzz_id'] == 4456887].sort_values('buzz_post_created_at')
def get_previous(X):
    prev = -1
    buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]    
    created_at = train[train['buzz_external_id'] == X]['buzz_post_created_at'].values[0]    
    tf = train[(train['buzz_post_buzz_id'] == buzzid)&(train['buzz_post_created_at'] < created_at)]
    try:
        prev = tf[-1:]['buzz_external_id'].values[0]
        return prev
    except:
        return -1

dr['previous_external_id'] = dr['buzz_external_id'].apply(get_previous)

In [20]:
dr[['previous_external_id', 'buzz_external_id', 'buzz_post_created_at', "buzz_post_buzz_id"]]

,previous_external_id,buzz_external_id,buzz_post_created_at,buzz_post_buzz_id
37197,-1,560066014088988_1228152523946997,2017-02-06 12:48:13.398595,4456887
36967,560066014088988_1228152523946997,491452930867938_1550064698340084,2017-02-06 18:26:53.590087,4456887


In [40]:
###parameter tuning using grid search
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
#                      ('clf', LinearSVC()),
#                      ('clf', RandomForestClassifier()),
#                      ('clf', LogisticRegression()),
                    ])
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)], 
              'vect__max_features': [10,100,1000, 10000],
              'tfidf__use_idf': (True, False),
              'tfidf__norm': ('l1', 'l2', None),
              'clf': [MultinomialNB(), LinearSVC(), RandomForestClassifier(), LogisticRegression(),],
#               'clf__alpha': (1e-2, 1e-3),
             }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_all['buzz_campaign_name'], X_all['class'])

In [41]:
gs_clf.best_params_

{'clf': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'tfidf__norm': 'l1',
 'tfidf__use_idf': True,
 'vect__max_features': 1000,
 'vect__ngram_range': (1, 3)}

In [42]:
gs_clf.best_score_

0.76712328767123283